In [5]:
import os
import json
from dotenv import load_dotenv

from pypdf import PdfReader
import gradio as gr

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [6]:
# defining the model
from openai import OpenAI

# Openai Model
openai = OpenAI()

# Gemini Model
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [7]:
## Read the text from file 

reader = PdfReader("./public/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

print(linkedin)

   
Contact
9004713782 (Mobile)
hiteshsolanki4623@gmail.com
www.linkedin.com/in/hitesh-
solanki-a058872a0 (LinkedIn)
Top Skills
vite
Algorithms
appwrite
Certifications
DevOps Engineer
gRPC Masterclass with Java &
Spring Boot [ Hands-On]
MEDDICC MEDDPICC Sales
training
Hitesh Solanki
Got an idea? Let's build it together!‎ | Software Engineer
Mumbai, Maharashtra, India
Summary
I'm a passionate problem solver and software developer from India
with expertise in multiple programming languages and a strong
understanding of data structures and algorithms. I love building
web applications and constantly challenge myself with LeetCode
problems to improve my problem-solving skills.
Experience
Freelencer
Freelance
April 2025 - Present (4 months)
Mumbai, Maharashtra, India
• Developed and implemented full-stack applications for various clients,
enhancing their technology capabilities and AI integration.  
• Collaborated with startups to create innovative solutions, leveraging cutting-
edge technol

In [8]:
with open("./public/profile.txt", "r", encoding="utf-8") as f:
    summary = f.read()

summary

'### **Hitesh Solanki – Software Engineer | AI-Driven Builder**\n\n📍 Mumbai, Maharashtra, India\n📞 9004713782\n📧 [hiteshsolanki4623@gmail.com](mailto:hiteshsolanki4623@gmail.com)\n🔗 [LinkedIn](https://www.linkedin.com/in/hitesh-solanki-a058872a0)\n\n---\n\n### **Summary**\n\nA passionate problem solver and full-stack software developer with strong fundamentals in data structures and algorithms. Enthusiastic about building scalable web applications and integrating AI. Actively solving LeetCode problems to enhance algorithmic skills.\n\n---\n\n### **Experience**\n\n**🛠️ Software Engineer (Strique AI)** | Jun 2025 – Present\n\n* Handled integrations like WhatsApp, SMS, Zerobounce, and Meta Pixel.\n* Managed CRM (Bird.com) campaigns, leads, and workflows.\n* Implemented analytics (Mixpanel, GA, MS Clarity) and PostgreSQL optimization.\n\n**🔧 Freelance Software Developer** | Apr 2025 – Jun 2025\n\n* Delivered full-stack solutions with AI integration for various clients.\n* Collaborated with

In [9]:
name = "Hitesh Solanki"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [11]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [12]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [13]:
# Create a Pydantic model for the Evaluation
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [14]:

evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [15]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [16]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [17]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [20]:

evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The agent provided an accurate and professional response to the user's question, based on the information provided. It also offered to answer other questions, which is appropriate.")

In [21]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [22]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
